# Efficient Yelp API Calls (Core)

**Goal:** 

Use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop.

In [1]:
!pip install YelpAPI

In [2]:
# imports

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# import json to read yelp api credentials
import json

# with open: yelp api credentials (save as variable)
with open('/Users/yang0108/.secret/yelp_api.json') as f:
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# import yelpapi class
from yelpapi import YelpAPI

# create instance with api key
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

In [5]:
# define location and term
LOCATION = 'Seattle, WA'
TERM = 'Korean'

In [6]:
# specify folder to save data in
FOLDER = 'Data/'

# make folder (if already exists, okay)
os.makedirs(FOLDER, exist_ok = True)

# specify JSON_FILE to save results to
JSON_FILE = FOLDER+f"results_in_progress_{LOCATION.split(',')[0]}_{TERM}.json"
JSON_FILE

'Data/results_in_progress_Seattle_Korean.json'

In [7]:
# define function to create and save an empty JSON file
# use parameter delete_if_exists = True if want to start over
# (if there is a previous file)
def create_json_file(JSON_FILE, delete_if_exists = False):
    
    # check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # if it DOES exist:
    if file_exists == True:
        
        # check if user wants to delete if exists
        if delete_if_exists == True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file.")
            
            # delete previous file and confirm it no longer exists
            os.remove(JSON_FILE)
            
            # recursive call to function after old file is deleted
            create_json_file(JSON_FILE, delete_if_exists = False)
    
    # if it does NOT exist:
    else:
        
        # inform user and save empty list
        print(f"[!] {JSON_FILE} not found. Saving empty list to new file.")
        
        # create any needed folders specified in the JSON_FILE path
        # get the folder name
        folder = os.path.dirname(JSON_FILE)
        
        # if JSON_FILE path included a folder:
        if len(folder) > 0:
            
            # create the folder
            os.makedirs(folder, exist_ok = True)
            
            # save empty list to start the json file
            with open(JSON_FILE, 'w') as f:
                json.dump([], f)

In [8]:
# create new empty json file (delete if already exists)
create_json_file(JSON_FILE, delete_if_exists = True)

[!] Data/results_in_progress_Seattle_Korean.json already exists. Deleting previous file.
[!] Data/results_in_progress_Seattle_Korean.json not found. Saving empty list to new file.


In [9]:
# load previous results and use length of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
previous_results

[]

In [10]:
# set offset based on previous results
n_results = len(previous_results)

print(f"{n_results} previous results found.")

0 previous results found.


In [11]:
# use search_query to perform api call
results = yelp_api.search_query(location = LOCATION,
                               term = TERM,
                               offset = n_results)

# how many results total?
total_results = results['total']
total_results

733

In [12]:
# how many results did we get in api call?
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
# calculate number of calls needed for total results
n_pages = math.ceil((results['total'] - n_results) / results_per_page)
n_pages

37

In [14]:
# loop to get all results by calling api multiple times
for i in tqdm_notebook(range(1, n_pages + 1)):
    
    # read in results in progress and check length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
        
    # save number of results to use as offset
    n_results = len(previous_results)
    
    # stop loop if called over 1000 times
    if (n_results + results_per_page) > 1000:
        print("Exceeded 1000 API calls. Stopping loop.")
        break
        
    # call api, use n_results as the offset
    results = yelp_api.search_query(location = LOCATION,
                                   term = TERM,
                                   offset = n_results)
    
    # append new results to previous results and save
    previous_results.extend(results['businesses'])
    
    # display previous results
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)
        
    # pause
    time.sleep(0.2)

  0%|          | 0/37 [00:00<?, ?it/s]

In [15]:
# convert results in progress json file to df
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,45iVzFsXpjUuiBw0Dhc_Qg,seoul-tofu-and-jjim-seattle,Seoul Tofu & Jjim,https://s3-media4.fl.yelpcdn.com/bphoto/2GgYXM...,False,https://www.yelp.com/biz/seoul-tofu-and-jjim-s...,132,"[{'alias': 'korean', 'title': 'Korean'}]",5.0,"{'latitude': 47.62224, 'longitude': -122.32064}","[delivery, pickup]",$$,"{'address1': '406 Broadway E', 'address2': Non...",+12062575642,(206) 257-5642,1130.239024
1,h1Q0Wkx5TUUZeVjJrXTwmQ,chan-seattle-seattle-2,Chan Seattle,https://s3-media2.fl.yelpcdn.com/bphoto/C5Iry3...,False,https://www.yelp.com/biz/chan-seattle-seattle-...,982,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 47.61312322818342, 'longitude': -...",[delivery],$$,"{'address1': '724 Pine St', 'address2': '', 'a...",+14256582626,(425) 658-2626,287.367244
2,ZHMHUTDxXhufLU-iFSNntg,hanok-seattle-2,Hanok,https://s3-media1.fl.yelpcdn.com/bphoto/j8YCeJ...,False,https://www.yelp.com/biz/hanok-seattle-2?adjus...,140,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 47.63383, 'longitude': -122.28028}",[delivery],$$,"{'address1': '4021 E Madison St', 'address2': ...",+12064023847,(206) 402-3847,4311.826008
3,nUkPOJ5p4E9U7e2DbOzzMw,meet-korean-bbq-seattle,Meet Korean BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/UwvDMA...,False,https://www.yelp.com/biz/meet-korean-bbq-seatt...,313,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 47.614352, 'longitude': -122.325214}","[delivery, pickup]",$$$$,"{'address1': '500 E Pike St', 'address2': None...",+12066952621,(206) 695-2621,361.544031
4,zIczxp6kS-2kyV937xBnAA,stone-korean-restaurant-seattle-2,Stone Korean Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/pZbb_4...,False,https://www.yelp.com/biz/stone-korean-restaura...,246,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"{'latitude': 47.62729, 'longitude': -122.34209}","[delivery, pickup]",$$,"{'address1': '900 Dexter Ave N', 'address2': '...",+12067172864,(206) 717-2864,1701.715591


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
728,F_MPDHiCK-NeQe0sWEbkuA,toshis-teriyaki-lynnwood,Toshi's Teriyaki,https://s3-media3.fl.yelpcdn.com/bphoto/Ixrsqf...,False,https://www.yelp.com/biz/toshis-teriyaki-lynnw...,47,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.0,"{'latitude': 47.80957, 'longitude': -122.32429}",[delivery],$,"{'address1': '20829 Hwy 99', 'address2': '', '...",+14257715320,(425) 771-5320,21717.313762
729,hub2g_k6kSzxm6z7WORLRA,teriyaki-town-lynnwood,Teriyaki Town,https://s3-media1.fl.yelpcdn.com/bphoto/DF9Jp5...,False,https://www.yelp.com/biz/teriyaki-town-lynnwoo...,156,"[{'alias': 'japanese', 'title': 'Japanese'}]",3.5,"{'latitude': 47.821778, 'longitude': -122.296413}",[delivery],$$,"{'address1': '4615 196th St', 'address2': 'Ste...",+14256735904,(425) 673-5904,23205.805037
730,0miEAgm2MbD0biNNzyq7XA,teriyaki-wok-express-lynnwood,Teriyaki Wok Express,https://s3-media2.fl.yelpcdn.com/bphoto/2QTeZQ...,False,https://www.yelp.com/biz/teriyaki-wok-express-...,115,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.0,"{'latitude': 47.8305082, 'longitude': -122.277...","[pickup, delivery]",$,"{'address1': '18500 33rd Ave W', 'address2': '...",+14257756760,(425) 775-6760,24348.283442
731,KqFc899W0u0PBZp0iopEaA,big-teriyaki-lynnwood,Big Teriyaki,https://s3-media3.fl.yelpcdn.com/bphoto/2AoRr4...,False,https://www.yelp.com/biz/big-teriyaki-lynnwood...,55,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 47.8645401682447, 'longitude': -1...",[delivery],$,"{'address1': '3625 148th St SW', 'address2': '...",+14256780072,(425) 678-0072,28049.458470
732,AQSRgIPv7B2szCeQF2LxLw,apple-teriyaki-federal-way,Apple Teriyaki,https://s3-media4.fl.yelpcdn.com/bphoto/NvvfbE...,False,https://www.yelp.com/biz/apple-teriyaki-federa...,36,"[{'alias': 'japanese', 'title': 'Japanese'}]",3.0,"{'latitude': 47.3563199, 'longitude': -122.30925}",[delivery],$,"{'address1': '27400 Pacific Hwy S', 'address2'...",+12538390110,(253) 839-0110,28725.671264


In [16]:
# check for duplicates of ids
final_df.duplicated(subset = 'id').sum()

0

In [17]:
# drop duplicates
final_df = final_df.drop_duplicates(subset = 'id')

# check
final_df.duplicated(subset = 'id').sum()

0

In [18]:
# save final results to csv
final_df.to_csv('Data/final_results_Seattle_Korean.csv',
               index = False)

# alternate option to zip if needed:
# final_df.to_csv('Data/final_results_Seattle_Korean.csv.gz',
#                 compression = 'gzip',
#                 index = False)